# Fetch Data

Fetches all Anilist data needed for practicing data visualization and machine learning.

In [1]:
from anilist_data import fetch as anilist

anilist.get_user(247578)['data']['User']

{'name': 'barrettotte',
 'createdAt': 1552085204,
 'statistics': {'anime': {'count': 605,
   'meanScore': 68.93,
   'standardDeviation': 15.9,
   'minutesWatched': 198339,
   'episodesWatched': 8119}}}